In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder\
.master("spark://192.168.2.70:7077") \
.appName("ludvigwesterholm_B")\
.config("spark.dynamicAllocation.enabled", True)\
.config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
.config("spark.shuffle.service.enabled", True)\
.config("spark.dynamicAllocation.executorIdleTimeout","30s")\
.config("spark.cores.max", 4)\
.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/23 11:53:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
#B.1 reading the csv
df = spark_session.read.csv("hdfs://192.168.2.70:9000/parking-citations.csv")
df.show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+--------------------+-----------+---------+---------+------------------+-----------------+--------------------+
|          _c0|                 _c1|       _c2|     _c3|        _c4|           _c5|              _c6| _c7| _c8|       _c9| _c10|                _c11| _c12|  _c13|          _c14|                _c15|       _c16|     _c17|     _c18|              _c19|             _c20|                _c21|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+--------------------+-----------+---------+---------+------------------+-----------------+--------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|     

In [7]:
#B.2 printing the schema
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)



In [11]:
#B.3 coutning number of rows
df.count()

13077725

In [14]:
#B.4 counting number of partitions
df.rdd.getNumPartitions()

type(df)

pyspark.sql.dataframe.DataFrame